<a href="https://colab.research.google.com/github/Tayes06/financial-anomaly-detection/blob/main/fraud_prevention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2
### Fraud detection in financial transactions
This notebook is deserved to continue work about fraud detection started in another one  
But here we'll focus on how to prevent insted of detect anomalies in financial transactions

Nous allons empêcher les fraudes AVANT que les transactions ne soient validées en évaluant un score de risque en temps réel.

### A. Loading and préaration of dataset
1. datast loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd

# Charger les données
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/bank-fraud-detection/paysim_dataset_for_prediction.csv")


##### Dataset enrichment

In [ ]:
np.random.seed(42)  # Pour avoir les mêmes résultats à chaque exécution

# Simuler le type d'appareil utilisé pour la transaction
device_types = ['Mobile', 'Desktop', 'ATM']
df['device_type'] = np.random.choice(device_types, size=len(df))

# Show devices distributions
print(df['device_type'].value_counts())

device_type
Desktop    2121683
ATM        2121611
Mobile     2119326
Name: count, dtype: int64


In [ ]:
cities = ['Douala', 'Yaoundé', 'Paris', 'Limoges', 'Abuja', 'Toronto', 'Guangzhou', 'Bafoussam', 'Abijan']
df['location'] = np.random.choice(cities, size=len(df))

# Visualisation des villes les plus fréquentes
print(df['location'].value_counts())

location
Abijan       708859
Abuja        707246
Yaoundé      707004
Paris        706873
Guangzhou    706746
Douala       706648
Limoges      706637
Bafoussam    706350
Toronto      706257
Name: count, dtype: int64


In [ ]:
def assign_time(step):
    if step % 24 < 6:
        return 'Nuit'
    elif step % 24 < 12:
        return 'Matin'
    elif step % 24 < 18:
        return 'Après-midi'
    else:
        return 'Soir'

df['time_of_day'] = df['step'].apply(assign_time)

# Vérifier la distribution des transactions par moment de la journée
print(df['time_of_day'].value_counts())

time_of_day
Après-midi    2689784
Soir          2365669
Matin         1194562
Nuit           112605
Name: count, dtype: int64


In [ ]:
from numba import jit

@jit(nopython=True)
def calculate_rapid_transactions(step_values):
    # Convert step_values to a NumPy array with float64 dtype
    # but first convert to list to ensure homogeneity
    step_values = np.array(list(step_values), dtype=np.float64)
    diff_values = np.diff(step_values)
    diff_values = np.concatenate((np.array([999]), diff_values))  # Handle first element using concatenate
    rapid_transactions = (diff_values < 1).astype(np.int64).cumsum()
    return rapid_transactions

# Vérifier si 'nameOrig' existe
if 'nameOrig' in df.columns:
    df['rapid_transactions'] = df.groupby('nameOrig')['step'].apply(lambda x: calculate_rapid_transactions(x.to_numpy()))
    # Fill NaN values with 0 before converting to int
    df['rapid_transactions'] = df['rapid_transactions'].fillna(0).astype(int)  # Convertir en int pour éviter les erreurs

print("✅ Colonne `rapid_transactions` créée avec succès !")

✅ Colonne `rapid_transactions` créée avec succès !


<ipython-input-16-75a9d2f1a26d>:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['rapid_transactions'] = df['rapid_transactions'].fillna(0).astype(int)  # Convertir en int pour éviter les erreurs


In [ ]:
# Supprimer les colonnes non pertinentes
df = df.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'])  # Identifiants inutiles

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'rapid_transactions']] = scaler.fit_transform(
    df[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'rapid_transactions']]
)

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   step                int64  
 1   type                object 
 2   amount              float64
 3   oldbalanceOrg       float64
 4   newbalanceOrig      float64
 5   oldbalanceDest      float64
 6   newbalanceDest      float64
 7   isFraud             int64  
 8   device_type         object 
 9   location            object 
 10  time_of_day         object 
 11  rapid_transactions  float64
dtypes: float64(6), int64(2), object(4)
memory usage: 582.5+ MB
None


2. Encoding of categorichal variables

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in ['type', 'time_of_day']:
    df[col] = encoder.fit_transform(df[col])

3. Normalization of amount and balance

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'rapid_transactions']] = scaler.fit_transform(
    df[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'rapid_transactions']]
)

In [ ]:
print(df.columns)  # Liste toutes les colonnes disponibles

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud', 'device_type',
       'location', 'time_of_day', 'rapid_transactions'],
      dtype='object')


4. Separate charachteristics x and the tagert y

In [ ]:
# Définition des variables explicatives et de la variable cible
X = df.drop(columns=['isFraud'])  # Caractéristiques
y = df['isFraud']  # Variable cible (1 = fraude, 0 = normal)

print("✅ Données prêtes pour l'entraînement !")

✅ Données prêtes pour l'entraînement !


## Spliting dataset into training and test set

In [ ]:
from sklearn.model_selection import train_test_split

# Séparation en entraînement (80%) et test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"📊 Entraînement : {X_train.shape[0]} transactions")
print(f"📊 Test : {X_test.shape[0]} transactions")

📊 Entraînement : 5090096 transactions
📊 Test : 1272524 transactions


### Training of the prediction model
We'll first start with Random Forest, then test XGBoost


*   Training of Random Forest



In [ ]:
print(X_train.dtypes)

step                    int64
type                    int64
amount                float64
oldbalanceOrg         float64
newbalanceOrig        float64
oldbalanceDest        float64
newbalanceDest        float64
device_type            object
location               object
time_of_day             int64
rapid_transactions    float64
dtype: object


In [ ]:
for col in X_train.select_dtypes(include=['object']).columns:
    print(f"📌 {col} : {X_train[col].unique()[:10]}")  # Affiche 10 valeurs uniques par colonne


📌 device_type : ['ATM' 'Desktop' 'Mobile']
📌 location : ['Guangzhou' 'Paris' 'Abijan' 'Bafoussam' 'Toronto' 'Abuja' 'Yaoundé'
 'Limoges' 'Douala']


In [ ]:
categorical_cols = X_train.select_dtypes(include=['object']).columns
print("📌 Colonnes catégoriques détectées :", categorical_cols)

📌 Colonnes catégoriques détectées : Index(['device_type', 'location'], dtype='object')


In [ ]:
#from sklearn.preprocessing import LabelEncoder

#encoder = LabelEncoder()
#for col in categorical_cols:
#    X_train[col] = encoder.fit_transform(X_train[col])
#    X_test[col] = encoder.transform(X_test[col])  # Encoder aussi X_test

In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Dictionnaire pour stocker les encodeurs de chaque colonne catégorique
encoders = {}

for col in categorical_cols:
    # Vérifier si la colonne contient déjà des nombres
    if X_train[col].dtype == 'object':
        le = LabelEncoder()
        X_train[col] = le.fit_transform(X_train[col])
        X_test[col] = le.transform(X_test[col])  # Encoder aussi X_test
        encoders[col] = le  # Sauvegarder l'encodeur uniquement pour les colonnes encodées

# Sauvegarder les encodeurs
joblib.dump(encoders, 'encoders.pkl')

print("✅ Encodeurs sauvegardés correctement !")


✅ Encodeurs sauvegardés correctement !


In [ ]:
print(X_train.dtypes)  # Vérifier que toutes les colonnes sont bien numériques


step                    int64
type                    int64
amount                float64
oldbalanceOrg         float64
newbalanceOrig        float64
oldbalanceDest        float64
newbalanceDest        float64
device_type             int64
location                int64
time_of_day             int64
rapid_transactions    float64
dtype: object


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialisation du modèle
model_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")

# Entraînement du modèle
model_rf.fit(X_train, y_train)

print("✅ Modèle Random Forest entraîné avec succès !")


✅ Modèle Random Forest entraîné avec succès !


## Test of the model on test set

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Prédictions
y_pred_rf = model_rf.predict(X_test)

# Évaluation
print("📊 Performance du modèle Random Forest :")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))
print(f"🎯 Précision : {accuracy_score(y_test, y_pred_rf):.4f}")

📊 Performance du modèle Random Forest :
[[1270868      13]
 [    388    1255]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.99      0.76      0.86      1643

    accuracy                           1.00   1272524
   macro avg       0.99      0.88      0.93   1272524
weighted avg       1.00      1.00      1.00   1272524

🎯 Précision : 0.9997


# Optimisation of the result using XGBoost
we'll now train model on XGBoost and compare results

In [ ]:
pip install xgboost

### Training of XGBoost model

In [ ]:
from xgboost import XGBClassifier

# Initialisation du modèle
model_xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, scale_pos_weight=5, random_state=42)

# Entraînement
model_xgb.fit(X_train, y_train)

# Prédictions
y_pred_xgb = model_xgb.predict(X_test)

# Évaluation
print("Performance du modèle XGBoost :")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print(f"Précision : {accuracy_score(y_test, y_pred_xgb):.4f}")

Performance du modèle XGBoost :
[[1270712     169]
 [    178    1465]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.90      0.89      0.89      1643

    accuracy                           1.00   1272524
   macro avg       0.95      0.95      0.95   1272524
weighted avg       1.00      1.00      1.00   1272524

Précision : 0.9997


Let's now compare and see the one most efficient

In [ ]:
print("Comparaison des précisions :")
print(f"- Random Forest : {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"- XGBoost : {accuracy_score(y_test, y_pred_xgb):.4f}")

Comparaison des précisions :
- Random Forest : 0.9997
- XGBoost : 0.9997


## Saving of XGBoost model for further use

In [ ]:
import joblib

# Sauvegarde du modèle XGBoost
joblib.dump(model_xgb, 'xgboost_fraud_model.pkl')

# Sauvegarde des outils de prétraitement
joblib.dump(encoder, 'encoders.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("Modèle XGBoost et outils de prétraitement sauvegardés !")

Modèle XGBoost et outils de prétraitement sauvegardés !


Let's load the just saved model and make some tests

In [ ]:
# Charger le modèle
model_loaded = joblib.load('xgboost_fraud_model.pkl')
encoder = joblib.load('encoders.pkl')
scaler = joblib.load('scaler.pkl')

print("Modèle chargé avec succès !")

Modèle chargé avec succès !


In [ ]:
# Charger les encodeurs, le scaler et le modèle
encoders = joblib.load('encoders.pkl')
scaler = joblib.load('scaler.pkl')
model_rf = joblib.load('model_rf.pkl')

# Fonction pour encoder une valeur de façon sécurisée
def safe_encode(column, value):
    if column in encoders:
        encoder = encoders[column]
        return encoder.transform([value])[0] if value in encoder.classes_ else encoder.transform(['UNKNOWN'])[0]
    return value  # Retourne la valeur si elle est déjà numérique


FileNotFoundError: [Errno 2] No such file or directory: 'model_rf.pkl'

# Tester une Nouvelle Transaction Avant Validation

Créons une fonction d'évaluation de risque

In [ ]:
def evaluer_risque(transaction):
    proba = model_loaded.predict_proba(transaction)[:, 1]  # Probabilité de fraude

    if proba[0] > 0.75:
        return f"🚨 Risque Élevé ({proba[0]:.2f}) → Transaction BLOQUÉE !"
    elif proba[0] > 0.5:
        return f"⚠️ Risque Modéré ({proba[0]:.2f}) → Vérification Supplémentaire demandée."
    else:
        return f"✅ Risque Faible ({proba[0]:.2f}) → Transaction Autorisée."


In [ ]:
print("📌 Nombre de colonnes attendues par `StandardScaler` :", scaler.n_features_in_)


📌 Nombre de colonnes attendues par `StandardScaler` : 6


Test d'une transaction suspecte

In [ ]:
# Charger les encodeurs
encoders = joblib.load('encoders.pkl')

# Fonction sécurisée d'encodage
def safe_encode(column, value):
    if column in encoders:
        encoder = encoders[column]
        return encoder.transform([value])[0] if value in encoder.classes_ else encoder.transform(['UNKNOWN'])[0]
    return value  # Retourne directement la valeur si elle n'a pas besoin d'être encodée

# Créer une transaction test
transaction_test = pd.DataFrame({
    'type': [safe_encode('type', 'TRANSFER')],
    'amount': [scaler.transform([[12000]])[0][0]],
    'oldbalanceOrg': [200],
    'newbalanceOrig': [0],
    'oldbalanceDest': [5000],
    'newbalanceDest': [20000],
    'device_type': [safe_encode('device_type', 'Desktop')],
    'location': [safe_encode('location', 'Nigeria')],
    'time_of_day': [safe_encode('time_of_day', 'Night')],
    'rapid_transactions': [9]
})

print(transaction_test.head())  # Vérifier que tout est bien encodé

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 1 features, but StandardScaler is expecting 6 features as input.